In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Auswerten eines Schach-Endspiels

Zum Vergleich, dass die berechneten Züge der KI eine gute Lösung für das Gewinnen einer Endspielsituation sind, wird das Programm Stockfish verwendet. Es stellt eine Schach-Engine dar, die nach einem zuvor definierten Muster optimale Schachzüge berechnet.

## Importe

Für die Durchführung der Tests und der Auswertung eines Schach-Endspiels gegen eine andere KI werden die hier stehenden Bibliotheken benötigt:

* ``chess``: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* ``stockfish``: Bibliothek, welche eine Python Schnittstelle für die Stockfish-Enginge anbietet.
* ``clear_output`` aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen
verwendet wird.
* ``display`` aus der IPython-display-Library: Wird zum grafischen Anzeigen der Schachbretter verwendet.
* ``json``: Wird zum Speichern der Ergebnisse verwendet.
* ``time``: Wird zum Verzögern von Ausgaben verwendet.

In [ ]:
import chess
import stockfish
from IPython.display import clear_output
from IPython.display import display
import json
import time

## Konfiguration

Zu Beginn müssen für den Vergleich unterschiedliche Variablen definiert werden. Diese beinhalten eine Fen, die für ein gegebenes Szenario überprüft werden soll, den Pfad zu der berechneten $S_n$ Sequenz und einen Pfad zu der Stockfish-Installation.

In [ ]:
FEN = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
S_N_FILE = "S_n_seq_12_01.json"
STOCKFISH_PATH = "./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die mit Hilfe des Notebooks ``calculation.ipynb`` bestimmt werden. Diese wurden bereits in einer JSON-Datei serialisiert abgespeichert. Für dieses Notebook werden alle FENs gelesen und daraus Board-Objekte erstellt. Dies geschieht mit der Funktion ``load_s_n_sequence``. Diese erhält den Dateipfad (``filepath``) als Parameter, um aus der Datei die $S_n$ Mengen auslesen zu können.

In [ ]:
def load_s_n_sequence(filepath):
    s_n_sequence_new = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_list = []
        for board in item:
            tmp_list.append(chess.Board(board))
        s_n_sequence_new.append(tmp_list)
    f.close()
    return s_n_sequence_new

## Die Züge für die KI ermitteln

Der erste Schritt, um einen Zug für die KI zu ermitteln, liegt im Finden der momentanen Spielsituation. Dies wird mit der Funktion ``find_board_in_sequence`` erreicht. Sie dursucht eine gegebene $S_n$ Sequenz nach einem übergebenen Schachbrett. Die Parameter hierfür lauten wie folgt:

* ``situation``: Das Board (als Objekt), welches gefunden werden soll.
* sequence: Die $S_n$ Sequenz, in welcher das Board gesucht wird.

Das Ergebnis der Funktion kann insgesamt zwei unterschiedliche Formen annehmen:
* Ein Tupel mit $S_n$ Index (z.B. $S_3$) und Board-Index (z.B. 100).Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
* Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Das Ergebniss wird mithilfe von dem anschließenden Algorithmus bestimmt:
1. Über die $S_n$ Sequenz iterieren.
2. Über jedes Board in einem spezifischem $S_n$ iterieren.
3. Das Board mit dem ``situation`` Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [ ]:
def find_board_in_sequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

Nachdem das Brett in einer $S_n$ Menge gefunden wurde, ist der nächste Schritt den passenden Zug für die KI zu bestimmen. Zu diesem Zweck wurde die Funktion ``find_next_move`` definiert. Diese berechnet für ein übergebenes Board den idealen Spielzug. Weiterhin wird aus Effizienzgründen schon die Position des nächsten Bretts in der $S_n$ Sequenz zurückgegeben. Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:

* ``curr_board``: Das Board, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich ``curr_board`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence``: Die Liste mit allen $S_n$

Als Ergebnis liefert die Funktion den nächsten Move oder den Wert -1 als Hinweis, dass kein Spielzug berechnet werden konnte.

Der Algorithmus dieser Funktion erstreckt sich folgender Art und Weise:
* Führe alle möglichen Moves durch, bis das Ergebnis-Board in $S_{n-1}$ gefunden wurde.
* Gebe den neuen ``s_index``, ``board_index`` und den gefundenen ``move`` zurück.

In [ ]:
def find_next_move(curr_board, s_index, s_n_sequence):
    STOCKFISH.set_fen_position(curr_board.fen())
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            _tmp, board_index_tmp = find_board_in_sequence(curr_board, [s_n_sequence[s_index - 1]])
            s_index_tmp = s_index - 1
            if board_index_tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp) + " - Board: " + str(board_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, board_index_tmp, move
            curr_board.pop()

        return -1, -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        s_index, board_index = find_board_in_sequence(curr_board, s_n_sequence[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index) + " - Board: " + str(board_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, board_index, move

## Vergleich zwischen Stockfish und KI

Für den Vergleich werden die Anzahlen der Züge zwischen Stockfish und der selbstgeschriebenen KI miteinander verglichen. 

Bei der Berechnung der Züge für die KI wird die Funktion ``calculate_all_moves`` verwendet. Diese berechnet in einer Schleife alle Moves ausgehend von einer Fen, bis die KI gewonnen hat. Die Berechnung verwendet hierzu folgende Parameter:

* ``fen``: Die Fen der Spielsituation, die berechnet werden soll.
* ``s_n_sequence``: Die Liste der $S_n$, welche für die Figurenkonstellation berechnet wurde.

Das Ergebnis der Ausführung ist eine Liste aller Moves, welche für das Board berechnet wurde oder der Wert None als Hinweis, dass kein Sieg erzielt werden konnte.

Die Liste wurde durch den anschließenden Algorithmus bestimmt:
* Einordnung der übergebenen Fen in der $S_n$ Sequenz finden.
* Solange $n > 0$ in einer Schleife den nächsten Spielzug berechnen.
* Die Liste der errechneten Spielzüge zurückgeben.

In [ ]:
def calculate_all_moves(fen, s_n_sequence):
    moves = []

    board = chess.Board(fen)

    s_index, board_index = find_board_in_sequence(board, s_n_sequence)

    while s_index > 0:
        curr_board = s_n_sequence[s_index][board_index].copy()
        s_index, board_index, next_move = find_next_move(curr_board, s_index, s_n_sequence)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

Nachdem eine Liste aller Züge für die KI bestimmt wurden, gilt dies gleichermaßen für die Berechnung von Stockfish umzusetzen. Hierfür erhält die Funktion ``stockfish_movelist`` den Parameter:
* ``fen``: Die FEN des Boardes, für welches alle Züge berechnet werden sollen.

Nach der Ausführung erhält man ebenso eine List von Spielzügen.

Die Liste wurde mit einem leicht abgeänderten Algorithmus bestimmt:
* Solange das Spiel nicht beendet ist einen weiteren Zug berechnen.
* Am Ende alle Züge zurückgeben.

In [ ]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves

Für die visuelle Darstellung der Züge wurde die Funktion ``show_movelist`` definiert. Diese zeigt auf einem ``chess.Board`` eine mitgegebene Liste von Zügen. Die Funktionsargumente der Funktion lauten:
* ``fen``: Die Fen des Boardes, für welches alle Züge dargestellt werden sollen.
* ``moves``: Die Liste der Spielzüge, welche dargestellt werden sollen.

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)

Der Vergleich zwischen der KI und Stockfish kann nun anhand der Länge der Listen durchgeführt werden. Außerdem können die Züge durch die Funktion ``show_movelist`` nachvollzogen werden.

In [ ]:
S_N_Sequence = load_s_n_sequence(S_N_FILE)
Moves = calculate_all_moves(FEN, S_N_Sequence)
Stockfish_Moves = stockfish_movelist(FEN)

if Moves is not None:
    print("AI needed " + str(len(Moves)) + " moves to beat Stockfish as Black.")
else:
    print("AI found no way to beat Black.")

print("Stockfish needed " + str(len(Stockfish_Moves)) + " moves to win against itself.")

In [ ]:
show_movelist(FEN, Moves)